In [3]:
!pip uninstall -y opencv-python-headless opencv-python opencv-contrib-python mediapipe numpy jax jaxlib jax-cuda12-plugin protobuf -q


!pip install --no-cache-dir \
  numpy==1.26.4 \
  "protobuf>=4.25.3,<5" \
  mediapipe==0.10.21 \
  opencv-contrib-python==4.11.0.86 -q



You can safely remove it manually.
You can safely remove it manually.


^C


In [14]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="jaxlib.plugin_support")

import platform, time, csv, os
import cv2, numpy as np, mediapipe as mp
from collections import deque

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# ---------- helpers ----------
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180.0 else angle

def torso_lean_deg(shoulder, hip):
    v = np.array([shoulder[0]-hip[0], shoulder[1]-hip[1]])
    up = np.array([0.0, -1.0])
    nv = np.linalg.norm(v)
    if nv < 1e-6: return 0.0
    cosang = np.clip(np.dot(v, up)/(nv*1.0), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosang)))

def open_camera_for_jupyter():
    osname = platform.system().lower()
    if osname.startswith("win"):
        backends = [cv2.CAP_DSHOW, cv2.CAP_MSMF, None]
    elif osname == "darwin":
        backends = [cv2.CAP_AVFOUNDATION, None]
    else:
        backends = [cv2.CAP_V4L2, None]
    for b in backends:
        for idx in (0,1,2):
            cap = cv2.VideoCapture(idx) if b is None else cv2.VideoCapture(idx, b)
            if cap.isOpened():
                cap.set(cv2.CAP_PROP_FRAME_WIDTH,  960)
                cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
                cap.set(cv2.CAP_PROP_FPS, 30)
                return cap
            cap.release()
    return None

# ---------- thresholds ----------
TOP_LOCK   = 165
BOTTOM_TH  = 95
LEAN_WARN  = 20
VIS_THR    = 0.7
QUALITY_POP_MS = 1500
# -------------------------------

def classify_rep(min_knee_angle, max_lean_deg):
    good_depth = min_knee_angle < (BOTTOM_TH - 5)
    ok_depth   = min_knee_angle < 115
    upright    = max_lean_deg < LEAN_WARN
    a_bit_lean = max_lean_deg < (LEAN_WARN + 5)

    if good_depth and upright:
        return "Good"
    if (good_depth and a_bit_lean) or (ok_depth and upright):
        return "Okay"
    return "Bad"

# ---------- setup ----------
cap = open_camera_for_jupyter()
if cap is None:
    raise RuntimeError("Could not open webcam. Close Zoom/Meet/OBS, check permissions.")

ok, probe = cap.read()
if not ok:
    cap.release()
    raise RuntimeError("Webcam opened but no frames received.")

h, w = probe.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_path = "smart_squat_fullscreen.mp4"
out = cv2.VideoWriter(video_path, fourcc, 20.0, (w, h))

counter, state = 0, "get_ready"
status_text, form_text = "", ""

knee_hist = deque(maxlen=5)
lean_hist = deque(maxlen=5)

curr_min_knee, curr_max_lean = 999.0, 0.0
show_quality_until, last_quality_text = 0.0, ""
last_quality_color = (0, 180, 0)

rep_records = []

csv_path = "rep_log.csv"
write_header = not os.path.exists(csv_path)
csv_file = open(csv_path, "a", newline="")
csv_writer = csv.writer(csv_file)
if write_header:
    csv_writer.writerow(["rep_index","min_knee_angle_deg","max_lean_deg","quality","timestamp"])

# full screen window
cv2.namedWindow("Smart Squat Trainer", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("Smart Squat Trainer", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    try:
        while True:
            ok, frame = cap.read()
            if not ok: break
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            try:
                lm = results.pose_landmarks.landmark
                LHIP, LKNE, LANK, LSHO = lm[23], lm[25], lm[27], lm[11]
                RHIP, RKNE, RANK, RSHO = lm[24], lm[26], lm[28], lm[12]

                l_hip, l_knee, l_ankle = [LHIP.x, LHIP.y], [LKNE.x, LKNE.y], [LANK.x, LANK.y]
                r_hip, r_knee, r_ankle = [RHIP.x, RHIP.y], [RKNE.x, RKNE.y], [RANK.x, RANK.y]
                l_sho, r_sho = [LSHO.x, LSHO.y], [RSHO.x, RSHO.y]

                l_knee_ang = calculate_angle(l_hip, l_knee, l_ankle)
                r_knee_ang = calculate_angle(r_hip, r_knee, r_ankle)
                knee = (l_knee_ang + r_knee_ang) / 2.0
                l_lean = torso_lean_deg(l_sho, l_hip)
                r_lean = torso_lean_deg(r_sho, r_hip)
                lean = (l_lean + r_lean) / 2.0

                hip_below_knee = (l_hip[1] > l_knee[1]) and (r_hip[1] > r_knee[1])
                is_visible = all(pt.visibility > VIS_THR for pt in [LHIP, LKNE, RHIP, RKNE, LSHO, RSHO])

                knee_hist.append(knee)
                lean_hist.append(lean)
                s_knee, s_lean = float(np.mean(knee_hist)), float(np.mean(lean_hist))

                # rep tracking
                if state == "get_ready":
                    if is_visible and s_knee > (TOP_LOCK - 5):
                        state, status_text = "ready", "READY"
                        curr_min_knee, curr_max_lean = 999.0, 0.0
                    else:
                        status_text = "STAND STRAIGHT TO START"

                elif state == "ready":
                    status_text = "DESCEND"
                    curr_min_knee = min(curr_min_knee, s_knee)
                    curr_max_lean = max(curr_max_lean, s_lean)
                    if s_knee < BOTTOM_TH or hip_below_knee:
                        state = "down"

                el
                if state == "down":
                    status_text = "DRIVE UP"
                    curr_min_knee = min(curr_min_knee, s_knee)
                    curr_max_lean = max(curr_max_lean, s_lean)
                    if s_knee > (TOP_LOCK - 5):
                        counter += 1
                        quality = classify_rep(curr_min_knee, curr_max_lean)
                        last_quality_text = f"REP {counter}: {quality}"
                        last_quality_color = {"Good":(0,200,0),"Okay":(0,165,255),"Bad":(0,0,255)}.get(quality,(255,255,255))
                        show_quality_until = time.time() + QUALITY_POP_MS/1000.0

                        csv_writer.writerow([counter, round(curr_min_knee,1), round(curr_max_lean,1), quality, round(time.time(),3)])
                        csv_file.flush()
                        rep_records.append((curr_min_knee, curr_max_lean, quality))
                        state, status_text = "ready", "REP COUNTED!"
                        curr_min_knee, curr_max_lean = 999.0, 0.0

                # live cues
                if not is_visible:
                    form_text = "MOVE INTO FRAME"
                elif s_lean > LEAN_WARN:
                    form_text = "CHEST UP"
                elif state in ("ready","down") and s_knee > 120:
                    form_text = "GO DEEPER"
                elif s_knee < BOTTOM_TH or hip_below_knee:
                    form_text = "GOOD DEPTH"
                else:
                    form_text = "GOOD FORM"

            except Exception:
                state, status_text, form_text = "get_ready", "NO BODY DETECTED", "—"

            # color logic
            if form_text in ("GOOD FORM", "GOOD DEPTH"): bar_color = (0, 180, 0)
            elif form_text == "GO DEEPER":               bar_color = (0, 0, 255)
            elif form_text == "CHEST UP":                bar_color = (0, 165, 255)
            else:                                        bar_color = (128, 0, 128)

            # UI boxes
            cv2.rectangle(image, (0, 0), (260, 78), (40, 40, 40), -1)
            cv2.putText(image, 'REPS', (15, 22), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1)
            cv2.putText(image, str(counter), (12, 64), cv2.FONT_HERSHEY_SIMPLEX, 1.6, (255,255,255), 2)
            cv2.putText(image, 'STATUS', (130, 22), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1)
            cv2.putText(image, status_text, (122, 62), cv2.FONT_HERSHEY_SIMPLEX, 0.95, (255,255,255), 2)

            # form bar
            cv2.rectangle(image, (260, 0), (max(960, image.shape[1]), 78), bar_color, -1)
            (tw, _), _ = cv2.getTextSize(form_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)
            text_x = 260 + (min(image.shape[1], 960) - 260 - tw) // 2
            cv2.putText(image, form_text, (max(260, text_x), 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2)

            # progress bar for squat depth
            progress = np.clip((TOP_LOCK - s_knee) / (TOP_LOCK - BOTTOM_TH), 0, 1)
            bar_w, bar_h = 25, int(progress * (image.shape[0]-100))
            cv2.rectangle(image, (image.shape[1]-60, image.shape[0]-50-bar_h), (image.shape[1]-35, image.shape[0]-50), (0,255,0) if progress>0.9 else (0,200,255), -1)
            cv2.rectangle(image, (image.shape[1]-60, image.shape[0]-50-(image.shape[0]-100)), (image.shape[1]-35, image.shape[0]-50), (255,255,255), 2)
            cv2.putText(image, f"{int(progress*100)}%", (image.shape[1]-85, image.shape[0]-60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

            # rep quality popup
            if time.time() < show_quality_until and last_quality_text:
                (qw, qh), _ = cv2.getTextSize(last_quality_text, cv2.FONT_HERSHEY_SIMPLEX, 1.1, 3)
                cx, y = image.shape[1] // 2, image.shape[0] - 60
                cv2.rectangle(image, (cx - qw//2 - 12, y - qh - 14), (cx + qw//2 + 12, y + 10), (30, 30, 30), -1)
                cv2.putText(image, last_quality_text, (cx - qw//2, y), cv2.FONT_HERSHEY_SIMPLEX, 1.1, last_quality_color, 3)

            # skeleton
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                )

            out.write(image)
            cv2.imshow("Smart Squat Trainer", image)
            key = cv2.waitKey(10) & 0xFF
            if key in (ord('q'), 27):
                break

    finally:
        cap.release()
        out.release()
        csv_file.close()
        cv2.destroyAllWindows()

        # summary
        if rep_records:
            avg_depth = np.mean([r[0] for r in rep_records])
            avg_lean = np.mean([r[1] for r in rep_records])
            good_count = sum(1 for r in rep_records if r[2]=="Good")
            print("\n=== SESSION SUMMARY ===")
            print(f"Total Reps: {len(rep_records)}")
            print(f"Average Depth (knee angle): {avg_depth:.1f}°")
            print(f"Average Lean: {avg_lean:.1f}°")
            print(f"Good Reps: {good_count} / {len(rep_records)} ({(good_count/len(rep_records))*100:.1f}%)")
            print(f"Video saved: {video_path}")
            print(f"CSV log: {csv_path}")
        else:
            print("No reps recorded.")



=== SESSION SUMMARY ===
Total Reps: 3
Average Depth (knee angle): 84.5°
Average Lean: 17.6°
Good Reps: 1 / 3 (33.3%)
Video saved: smart_squat_fullscreen.mp4
CSV log: rep_log.csv


KeyboardInterrupt: 